In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import jieba
import os
import pickle
import math
from tqdm.notebook import tqdm
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path
from torch import Tensor
from torch.nn import Transformer

In [2]:
# Load translation dataset from huggingface
os.environ['HF_DATASETS_OFFLINE'] = '1' # Comment this line if you need to download the dataset from huggingface
dataset = load_dataset('wmt19', 'zh-en')
print(dataset)
SRC_LANGUAGE = 'zh'
TGT_LANGUAGE = 'en'

Found cached dataset wmt19 (D:/Archives/HuggingfaceCache/datasets/wmt19/zh-en/1.0.0/29e210fae5690e843cae5dc43b53db36c4e02f927db50cd5235a22ab42dde90a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 25984574
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3981
    })
})


In [3]:
# Hyper-parameters
SUBSET_SIZE = 50000
VOCAB_MIN_FREQ = 10

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
SPECIAL_SYMBOLS = ['<UNK>', '<PAD>', '<BOS>', '<EOS>']
VOCAB_PATH = './Data/Vocab.pkl'

BATCH_SIZE = 64
LEARNING_RATE = 0.0001
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
DROPOUT = 0.2

In [4]:
# Make token transformers that can be used to tokenize text into list of tokens(words).
# I use the basic english tokenizer from torchtext for English.
# And use jieba for Chinese.
token_transform = {}
token_transform[TGT_LANGUAGE] = get_tokenizer('basic_english')
token_transform[SRC_LANGUAGE] = lambda text: ([x for x in jieba.lcut(text) if x not in {' ', '\t'}])

# test_sentence_zh = '但后来他们逐渐意识到所探测到的信号可能完全来源于星际尘埃。'
# test_sentence_en = 'It was later realized that the signal they had detected could be entirely attributed to interstellar dust.'
# assert token_transform[SRC_LANGUAGE](test_sentence_zh) == ['但', '后来', '他们', '逐渐', '意识', '到', '所', '探测', '到', '的', '信号', '可能', '完全', '来源于', '星际', '尘埃', '。']
# assert token_transform[TGT_LANGUAGE](test_sentence_en) == ['it', 'was', 'later', 'realized', 'that', 'the', 'signal', 'they', 'had', 'detected', 'could', 'be', 'entirely', 'attributed', 'to', 'interstellar', 'dust', '.']

# Yield tokens from data iterator. For each data {'en':'...', 'zh':'...'} in data_iter, yield a list of tokens in corresponding language using token_transform.
def yield_tokens(data_iter, language):
    for data in data_iter:
        yield token_transform[language](data[language])

# Build the vocabulary that can be used to encode token(word) into integer.
if Path(VOCAB_PATH).exists():
    # If we already have the vocab, load it
    with open(VOCAB_PATH, 'rb') as f:
        vocab_transform = pickle.load(f)
else:
    # Otherwise, build the vocab
    vocab_transform = {}
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        train_iter = iter(dataset['train'][:SUBSET_SIZE]['translation'])
        vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln), min_freq=VOCAB_MIN_FREQ, specials=SPECIAL_SYMBOLS, special_first=True)
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        vocab_transform[ln].set_default_index(UNK_IDX)
    with open(VOCAB_PATH, 'wb') as f:
        pickle.dump(vocab_transform, f)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
print(f'Vocab({TGT_LANGUAGE}) Size: {TGT_VOCAB_SIZE}')
print(f'Vocab({SRC_LANGUAGE}) Size: {SRC_VOCAB_SIZE}')

Vocab(en) Size: 7884
Vocab(zh) Size: 7751


In [5]:
# Now we need to make the real tokenizer that can tokenize a string of text into a sequence of integer tensors.

# Helper function that passes a string into a list of transforms.
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# Helper function that adds a BOS and EOS token to a list of tokens. E.g. [BOS_IDX, 5, 7, ..., 456, EOS_IDX]
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# The real tokenizer.
tokenizer = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    tokenizer[ln] = sequential_transforms(token_transform[ln], vocab_transform[ln], tensor_transform)

# print(tokenizer[TGT_LANGUAGE](test_sentence_en))
# Output:
# tensor([   2,   17,   38,  660, 3413,   12,    5, 3510,   37,  103,    0,   60, 18, 1667, 4340,    7,    0, 5568,    6,    3])

# print(tokenizer[SRC_LANGUAGE](test_sentence_zh))
# Output:
# tensor([   2,   13, 2221,   36,  843, 1092,   47,   49,    0,   47,    4, 1358, 37,  361, 3490,    0,    0,    6,    3])

In [6]:

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(tokenizer[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(tokenizer[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

class WMT19Dataset(Dataset):
    def __init__(self, dataset, subset_size = None):
        self.dataset = dataset
        self.subset_size = subset_size

    def __len__(self):
        if self.subset_size is None:
            return len(self.dataset)
        return self.subset_size

    def __getitem__(self, idx):
        return self.dataset[idx]['translation'][SRC_LANGUAGE], self.dataset[idx]['translation'][TGT_LANGUAGE]
    
train_dataset = WMT19Dataset(dataset['train'], SUBSET_SIZE)
valid_dataset = WMT19Dataset(dataset['validation'])

print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(valid_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

Train dataset size: 50000
Validation dataset size: 3981


In [7]:
# Transformer implementation from https://pytorch.org/tutorials/beginner/translation_transformer.html

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
    
    
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [8]:
transformer = Seq2SeqTransformer(num_encoder_layers=NUM_ENCODER_LAYERS, num_decoder_layers=NUM_DECODER_LAYERS, emb_size=EMB_SIZE, nhead=NHEAD, src_vocab_size=SRC_VOCAB_SIZE, tgt_vocab_size=TGT_VOCAB_SIZE, dim_feedforward=FFN_HID_DIM, dropout=DROPOUT)
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
print(f'Model params: {sum(p.numel() for p in transformer.parameters() if p.requires_grad)/1000000:.2f}M', transformer)
transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.AdamW(transformer.parameters(), lr=LEARNING_RATE)

Model params: 24.67M Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=512, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerDecoderLayer(
          (self_a

In [9]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    
    total_steps = 0
    for src, tgt in tqdm(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        total_steps += 1
    return losses / total_steps


def evaluate(model):
    model.eval()
    losses = 0

    total_steps = 0
    for src, tgt in valid_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        total_steps += 1
    return losses / total_steps

In [10]:
from timeit import default_timer as timer

for epoch in range(NUM_EPOCHS):
    start_time = timer()
    print("-" * 40)
    print("Start epoch {}/{}".format(epoch + 1, NUM_EPOCHS))
    print("-" * 40)
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    torch.save(transformer.state_dict(), './Model/Transformer.pth')
    print((f"Finished epoch: {epoch + 1}| Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s"))

----------------------------------------
Start epoch 1/15
----------------------------------------


  0%|          | 0/782 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\indie\AppData\Local\Temp\jieba.cache
Loading model cost 0.341 seconds.
Prefix dict has been built successfully.
e:\Software\Miniconda\envs\ml-torch\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
e:\Software\Miniconda\envs\ml-torch\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Finished epoch: 1| Train loss: 5.796, Val loss: 5.287, Epoch time = 61.795s
----------------------------------------
Start epoch 2/15
----------------------------------------


  0%|          | 0/782 [00:00<?, ?it/s]

...

Finished epoch: 13| Train loss: 2.687, Val loss: 4.209, Epoch time = 75.306s
----------------------------------------
Start epoch 14/15
----------------------------------------


  0%|          | 0/782 [00:00<?, ?it/s]

Finished epoch: 14| Train loss: 2.597, Val loss: 4.207, Epoch time = 74.973s
----------------------------------------
Start epoch 15/15
----------------------------------------


  0%|          | 0/782 [00:00<?, ?it/s]

Finished epoch: 15| Train loss: 2.514, Val loss: 4.216, Epoch time = 74.523s


In [11]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def beam_search(model, src, src_mask, max_len, start_symbol, beam_size=3):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    beam = [(ys, 0)]
    for i in range(max_len-1):
        candidates = []
        for snt, score in beam:
            if snt[0][-1] == EOS_IDX:
                candidates.append((snt, score))
                continue
            memory = memory.to(DEVICE)
            tgt_mask = (generate_square_subsequent_mask(snt.size(0))
                        .type(torch.bool)).to(DEVICE)
            out = model.decode(snt, memory, tgt_mask)
            out = out.transpose(0, 1)
            prob = model.generator(out[:, -1])
            topk = torch.topk(prob, beam_size)
            for next_word, word_score in zip(topk.indices[0], topk.values[0]):
                next_word = next_word.item()
                word_score = word_score.item()
                candidate = (torch.cat([snt,
                                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0),
                             score + word_score)
                candidates.append(candidate)
        beam = sorted(candidates, key=lambda x: x[1], reverse=True)[:beam_size]
    return beam[0][0]

def translate(model: torch.nn.Module, src_sentence: str, use_beam_search=False):
    model.eval()
    src = tokenizer[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    if use_beam_search:
        tgt_tokens = beam_search(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, beam_size=5).flatten()
    else:
        tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<BOS>", "").replace("<EOS>", "")

In [15]:
NUM_TEST = 20
for i in range(NUM_TEST):
    src, truth = valid_dataset[i]
    translation = translate(transformer, src, True)
    print('-'*40)
    print(f'Src: {src}')
    print(f'Truth: {truth}')
    print(f'Translation: {translation}')
    print('-'*40)

----------------------------------------
Src: 上周，古装剧《美人私房菜》临时停播，意外引发了关于国产剧收视率造假的热烈讨论。
Truth: Last week, the broadcast of period drama “Beauty Private Kitchen” was temporarily halted, and accidentally triggered heated debate about faked ratings of locally produced dramas.
Translation:  last week , the third plenum of the <UNK> <UNK> composite index , has been a <UNK> discussion of discussion about <UNK> .  . ”  . 
----------------------------------------
----------------------------------------
Src: 民权团体针对密苏里州发出旅行警告
Truth: Civil rights group issues travel warning for Missouri
Translation:  <UNK> groups have warned in <UNK> .      
----------------------------------------
----------------------------------------
Src: 由于密苏里州的歧视性政策和种族主义袭击，美国有色人种促进协会 (NAACP) 向准备前往密苏里州出游的有色人群发出旅行警告。
Truth: The National Association for the Advancement of Colored People has put out an alert for people of color traveling to Missouri because of the state's discriminatory policies and racist attacks.
Translation: